In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [3]:
busstops = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/busstops_norway.csv')
grunnkrets_age = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_age_distribution.csv')
grunnkrets_household = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_income_households.csv')
grunnkrets_norway = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/plaace_hierarchy.csv')
sample_submission = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/sample_submission.csv')
stores_extra = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_extra.csv')
stores_test = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_test.csv')
stores_train = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_train.csv')
simens_df = pd.read_csv("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/notebooks/simen/simens_dataframe-1.csv")

In [4]:
def impude_NaN (stores_df, grunnkrets_df, NaN_string ): 
    geo_df = grunnkrets_df[grunnkrets_df["year"] == 2016]
    geo_df2 = geo_df.drop("year", axis = 1)
    merged_df = stores_df.merge(geo_df2, how = "left", on = "grunnkrets_id")
    NaN_df = merged_df[merged_df[NaN_string].isna()]
    split_df = merged_df[merged_df[NaN_string].notna()]

    mat = cdist(NaN_df[['lat', 'lon']],
                split_df[['lat', 'lon']], metric='euclidean')

    new_df = pd.DataFrame(mat, index= NaN_df['grunnkrets_id'], columns=split_df['grunnkrets_id'])

    grunnkrets_id = NaN_df.grunnkrets_id
    closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)

    closest_df_with_distance = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values, "distance": distance.values})
    closest_df = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values})

    df_with_values_from_valid_id = split_df[split_df["grunnkrets_id"].isin(closest.values)]
    df_with_values_from_valid_id_removed_duplicates = df_with_values_from_valid_id.drop_duplicates(subset = ["grunnkrets_id"])

    df_valid_geo_data = df_with_values_from_valid_id_removed_duplicates.iloc[:,12:]
    df_with_only_gk_id = df_with_values_from_valid_id_removed_duplicates[["grunnkrets_id"]]
    df_list = [df_with_only_gk_id,df_valid_geo_data ]
    df_valid_geo_data_and_id = pd.concat(df_list, axis=1)

    df_without_nan = NaN_df.iloc[:,:12]

    df_including_closest_valid_id = df_without_nan.reset_index().merge(closest_df, how = "left", on = "grunnkrets_id").set_index("index")

    df_impuded = df_including_closest_valid_id.reset_index().merge(df_valid_geo_data_and_id, how="left", left_on ="closest_valid_id", right_on="grunnkrets_id").set_index("index")
    df_impuded_without_duplicates = df_impuded.drop_duplicates(subset="store_id")

    new_df_impuded = df_impuded_without_duplicates.drop(["closest_valid_id", "grunnkrets_id_y"], axis = 1).rename(columns= {"grunnkrets_id_x": "grunnkrets_id"})


    impuded = pd.concat([split_df, new_df_impuded])

    return impuded




In [5]:
def impude_NaN_test_set(test_df, grunnkrets_df, NaN_string): 
    geo_df = grunnkrets_df[grunnkrets_df["year"] == 2016]
    geo_df2 = geo_df.drop("year", axis = 1)
    merged_df = test_df.merge(geo_df2, how = "left", on = "grunnkrets_id")
    NaN_df = merged_df[merged_df[NaN_string].isna()]
    split_df = merged_df[merged_df[NaN_string].notna()]

    mat = cdist(NaN_df[['lat', 'lon']],
                split_df[['lat', 'lon']], metric='euclidean')

    new_df = pd.DataFrame(mat, index= NaN_df['grunnkrets_id'], columns=split_df['grunnkrets_id'])

    grunnkrets_id = NaN_df.grunnkrets_id
    closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)

    closest_df_with_distance = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values, "distance": distance.values})
    closest_df = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values})

    df_with_values_from_valid_id = split_df[split_df["grunnkrets_id"].isin(closest.values)]
    df_with_values_from_valid_id_removed_duplicates = df_with_values_from_valid_id.drop_duplicates(subset = ["grunnkrets_id"])

    df_valid_geo_data = df_with_values_from_valid_id_removed_duplicates.iloc[:,11:]
    df_with_only_gk_id = df_with_values_from_valid_id_removed_duplicates[["grunnkrets_id"]]
    df_list = [df_with_only_gk_id,df_valid_geo_data ]
    df_valid_geo_data_and_id = pd.concat(df_list, axis=1)

    df_without_nan = NaN_df.iloc[:,:11]

    df_including_closest_valid_id = df_without_nan.reset_index().merge(closest_df, how = "left", on = "grunnkrets_id").set_index("index")

    df_impuded = df_including_closest_valid_id.reset_index().merge(df_valid_geo_data_and_id, how="left", left_on ="closest_valid_id", right_on="grunnkrets_id").set_index("index")
    df_impuded_without_duplicates = df_impuded.drop_duplicates(subset="store_id")

    new_df_impuded = df_impuded_without_duplicates.drop(["closest_valid_id", "grunnkrets_id_y"], axis = 1).rename(columns= {"grunnkrets_id_x": "grunnkrets_id"})


    impuded = pd.concat([split_df, new_df_impuded])

    return impuded

In [6]:
stores_gk_impuded = impude_NaN(stores_train, grunnkrets_norway, NaN_string="district_name")
stores_house_impuded = impude_NaN(stores_train, grunnkrets_household, "singles")
stores_age_impuded = impude_NaN(stores_train, grunnkrets_age, NaN_string="age_0")
stores_test_impuded = impude_NaN_test_set(stores_test, grunnkrets_norway, NaN_string="district_name")




In [7]:
stores_gk_impuded2 = stores_gk_impuded.set_index("store_id").drop(["geometry"], axis = 1)
stores_test_impuded2 = stores_test_impuded.set_index("store_id").drop(["geometry"], axis = 1)

In [14]:
stores_extra

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name
0,911669196-973140302-27020,2016,BURGER KING STRANDGATEN,1.1.1.0,Hamburger restaurants,12010115,STRANDGATEN 5,60.393979,5.323851,BURGER KING,NaN
1,913155726-992998792-5832,2016,BURGER KING JESSHEIM STORSENTER,1.1.1.0,Hamburger restaurants,2350205,STORGATA 6,60.142760,11.171834,BURGER KING,Jessheim Storsenter
2,988252905-981108604-47887,2016,BURGER KING LANGNES,1.1.1.0,Hamburger restaurants,19020419,HULDERVEGEN 6,69.671483,18.920483,BURGER KING,NaN
3,995194546-891354622-45262,2016,BURGER KING MAGNETEN LEVANGER,1.1.1.0,Hamburger restaurants,17190701,NaN,63.732791,11.281785,BURGER KING,Magneten
4,881339692-979954964-3203,2016,BURGER KING ULLEVAAL STADION,1.1.1.0,Hamburger restaurants,3014511,SOGNSVEIEN 75,59.948558,10.732823,BURGER KING,Amfi Ullevaal
...,...,...,...,...,...,...,...,...,...,...,...
28679,931186744-982303729-41793,2016,COOP PRIX FRAMNES,2.8.1.0,Grocery stores,7061108,VESTERØYVEIEN 27,59.123102,10.236432,COOP PRIX,NaN
28680,931186744-983814964-6308,2016,KIWI MINI PRIS VALENTINLYST,2.8.1.0,Grocery stores,16012118,ANDERS ESTENSTADS VEG 4,63.424115,10.442566,KIWI NORGE,Valentinlyst Senter
28681,931186744-990018995-100187,2016,EXTRA MAASESKJÆRET,2.8.1.0,Grocery stores,12010617,SANDVIKSVEIEN 94,60.414032,5.320277,COOP EXTRA,NaN
28682,931186744-971707283-20794,2016,EXTRA MANDAL,2.8.1.0,Grocery stores,10020110,KASTELLGATA 14,58.022656,7.451600,COOP EXTRA,NaN


In [25]:
merged_df = stores_train.merge(stores_extra, how = "inner", on = "grunnkrets_id")
merged_df2 = merged_df.drop(["year_x", "year_y"], axis = 1)
merged_df2

,store_id_x,store_name_x,plaace_hierarchy_id_x,sales_channel_name_x,grunnkrets_id,address_x,lat_x,lon_x,chain_name_x,mall_name_x,revenue,store_id_y,store_name_y,plaace_hierarchy_id_y,sales_channel_name_y,address_y,lat_y,lon_y,chain_name_y,mall_name_y
0,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,859092152-988619698-88165,CHINA PALACE BIT FOR BIT,1.1.4.0,Chinese restaurants,SCHWENCKEGATA 6,59.743845,10.208353,NaN,NaN
1,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,880896822-980928365-46579,EAST-LAKE RESTAURANT,1.1.6.2,Restaurants,NaN,59.744230,10.206481,NaN,NaN
2,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,988434809-988464635--988464635,BUDDY ROCK AS,1.3.2.0,Pubs,AMTMAND BLOMS GATE 4,59.744069,10.206404,NaN,NaN
3,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,983853471-983859313-52104,PHONES 4 YOU AS,2.3.2.0,Telecom shops,NEDRE STORGATE 6,59.742854,10.206751,NaN,Magasinet Drammen
4,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,916224117-916255780-798999,RAJANFASHION B 21,2.4.6.0,Clothing stores,NEDRE STORGATE 6,59.742854,10.206751,NaN,Magasinet Drammen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376230,995358352-914709822-734652,CUREBERRY AS,2.8.10.0,Health food stores,2191022,MARIES VEI 20,59.894652,10.582810,NaN,NaN,0.973,914791723-996112446-152354,ISS FACILITY SERVICES AS,1.1.8.0,Cafeterias,MARIES VEI 20,59.894652,10.582810,NaN,NaN
376231,915643728-915813496-782249,SAGA HELSE,2.8.10.0,Health food stores,1052205,KRISTIAN WILHELMSENS VEI 3,59.291973,11.096977,NaN,NaN,1.856,983162886-983171834-15749,THØGERBOATS AS,2.6.3.2,Sporting goods stores,KIRKEVEIEN 39,59.292474,11.087211,NaN,NaN
376232,915643728-915813496-782249,SAGA HELSE,2.8.10.0,Health food stores,1052205,KRISTIAN WILHELMSENS VEI 3,59.291973,11.096977,NaN,NaN,1.856,912643689-912680533-649916,IRENS HÅRPALETT,3.2.1.0,Hairdressers,KIRKEVEIEN 20,59.293401,11.095468,NaN,NaN
376233,977056934-972208477-43421,VAALAND SPESIALBUTIKKEN,2.8.11.2,Beer and soda shop,11210203,HETLANDSGATA 32,58.731908,5.652061,NaN,NaN,16.700,939964630-973134426-10309,KIWI MINIPRIS ROSSELAND,2.8.1.0,Grocery stores,ROSSELANDSVEGEN,58.732531,5.660985,KIWI NORGE,NaN


In [30]:
chain_notNa = stores_train[stores_train["chain_name"].notna()]
chain_na = stores_train[stores_train["chain_name"].isna()]

chain_extra_notNa = stores_extra[stores_extra["chain_name"].notna()]
chain_extra_na = stores_extra[stores_extra["chain_name"].isna()]

In [46]:
chain_notNa

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528
...,...,...,...,...,...,...,...,...,...,...,...,...
12842,914490251-914516374-727898,2016,SUNKOST ÅLGÅRD,2.8.10.0,Health food stores,11220101,SKOGHOLTET 23,58.770232,5.846296,SUNKOST BUTIKK,Amfi Ålgård,3.576
12845,980753476-980895009-14991,2016,LIFE STRAUME,2.8.10.0,Health food stores,3010104,NaN,59.911677,10.747856,LIFE,Arkaden Shopping,0.004
12846,916035527-916090978-793078,2016,GULATING ØLUTSALG GAMLE FREDRIKSTAD,2.8.11.2,Beer and soda shop,1061105,FÆRGEPORTGATEN 78,59.204107,10.954810,GULATING GRUPPEN,NaN,5.813
12855,917921733-917982368-868081,2016,GULATING ØLUTSALG CC GJØVIK,2.8.11.2,Beer and soda shop,5020406,NaN,60.799991,10.693635,GULATING GRUPPEN,CC Gjøvik,1.816


In [63]:
chain_na.nlargest(40, "revenue")

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
6063,976754360-971751266-224,2016,OSLO SPORTSLAGER,2.6.3.2,Sporting goods stores,3010208,TORGGATA 20,59.916298,10.751809,NaN,NaN,219.599
4667,995738279-995744120-128728,2016,JOLLYROOM,2.4.5.0,Children’s clothing and equipment stores,7061101,URANIENBORGVEIEN 4,59.130997,10.246370,NaN,NaN,196.687
6736,967682535-974371987-357577,2016,TELEPERFORMANCE NORGE AS,2.7.1.0,Other stores,8051106,PORSELENSVEGEN 12,59.140137,9.637963,NaN,Porselensfabrikken,164.513
3618,986907890-986919058-366295,2016,BLÅ BOLIG AS,2.1.6.0,Construction stores,18120106,NaN,65.311356,12.166223,NaN,NaN,114.382
6882,989331388-989343769-131135,2016,LOUIS VUITTON,2.7.3.0,Travel accessories stores,3010103,NEDRE SLOTTSGATE 13,59.912189,10.742382,NaN,NaN,110.400
11627,975807371-975825639-390482,2016,FJØSSYSTEMER BYGG AS,3.3.2.0,Construction workers,5200205,LOSNABAKKAN 50,61.387670,10.247285,NaN,NaN,98.935
398,991949283-993310190-115693,2016,UTSIKTEN RESTAURANTDRIFT AS,1.1.6.1,Eateries,12010116,TORGET 2,60.395178,5.325278,NaN,NaN,81.609
5556,951282774-974186136-49528,2016,TATLER AS,2.4.6.0,Clothing stores,3010802,BOGSTADVEIEN 2,59.924056,10.725077,NaN,NaN,71.605
53,999276032-985152012-278955,2016,DOMSTEIN FISH AS,1.1.2.0,Sushi and wok restaurants,14390306,SJØGATA,61.934788,5.114448,NaN,Måløy Brygge,70.000
6564,994460242-992636467-209727,2016,ARTISTI TALE AS,2.7.1.0,Other stores,8150505,FRYDENSBORGVEIEN 4,58.870247,9.391031,NaN,Kalstad-senteret,68.651


In [36]:
chain_extra_notNa

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name
0,911669196-973140302-27020,2016,BURGER KING STRANDGATEN,1.1.1.0,Hamburger restaurants,12010115,STRANDGATEN 5,60.393979,5.323851,BURGER KING,NaN
1,913155726-992998792-5832,2016,BURGER KING JESSHEIM STORSENTER,1.1.1.0,Hamburger restaurants,2350205,STORGATA 6,60.142760,11.171834,BURGER KING,Jessheim Storsenter
2,988252905-981108604-47887,2016,BURGER KING LANGNES,1.1.1.0,Hamburger restaurants,19020419,HULDERVEGEN 6,69.671483,18.920483,BURGER KING,NaN
3,995194546-891354622-45262,2016,BURGER KING MAGNETEN LEVANGER,1.1.1.0,Hamburger restaurants,17190701,NaN,63.732791,11.281785,BURGER KING,Magneten
4,881339692-979954964-3203,2016,BURGER KING ULLEVAAL STADION,1.1.1.0,Hamburger restaurants,3014511,SOGNSVEIEN 75,59.948558,10.732823,BURGER KING,Amfi Ullevaal
...,...,...,...,...,...,...,...,...,...,...,...
28679,931186744-982303729-41793,2016,COOP PRIX FRAMNES,2.8.1.0,Grocery stores,7061108,VESTERØYVEIEN 27,59.123102,10.236432,COOP PRIX,NaN
28680,931186744-983814964-6308,2016,KIWI MINI PRIS VALENTINLYST,2.8.1.0,Grocery stores,16012118,ANDERS ESTENSTADS VEG 4,63.424115,10.442566,KIWI NORGE,Valentinlyst Senter
28681,931186744-990018995-100187,2016,EXTRA MAASESKJÆRET,2.8.1.0,Grocery stores,12010617,SANDVIKSVEIEN 94,60.414032,5.320277,COOP EXTRA,NaN
28682,931186744-971707283-20794,2016,EXTRA MANDAL,2.8.1.0,Grocery stores,10020110,KASTELLGATA 14,58.022656,7.451600,COOP EXTRA,NaN


In [56]:
chain_grouped = chain_notNa.groupby(["chain_name"], as_index=False)["store_id"].count().sort_values(by = ["store_id"], ascending = False)
chain_grouped_list = chain_grouped.chain_name.values.tolist()

chain_grouped_list

['REMA FRANCHISE NORGE',
 'JOKER',
 'MIX',
 'CIRCLE K DETALJIST',
 'BUNNPRIS',
 'INTERFLORA NORGE SA',
 'SPAR',
 'YX NORGE',
 'KIWI NORGE',
 'NÆRBUTIKKEN',
 'SPORT 1 BUTIKKENE',
 'C OPTIKK',
 'ESSO NORGE',
 'PIZZABAKEREN INVEST',
 'SPECSAVERS NORWAY',
 'FARGERIKE',
 'CHOICE HOTELS & RESORTS',
 'BJØRKLUND NORGE',
 'MX SPORT NORGE',
 'MØBELRINGEN',
 'G-SPORT',
 'NORSKE SHELL',
 'TILBORDS',
 'FLORISS DRIFT',
 'KITCHN BUTIKKDRIFT',
 'DITT APOTEK',
 'COOP EXTRA',
 'MATKROKEN',
 'BEST STASJON',
 'BYGGTORGET',
 'EXPERT',
 'EUROPRIS',
 'INTERSPORT NORGE',
 'TELENORBUTIKKEN',
 'SUNKOST BUTIKK',
 'FAGMØBLER NORGE',
 'INTEROPTIK',
 'STADION',
 'B-YOUNG',
 'JERNIA DETALJ',
 'EURO SKO NORGE',
 'BOHUS',
 'PEPPES PIZZA',
 'EURONICS',
 'MENY',
 'SCANDITOY',
 'ALLIANCE OPTIKK',
 'JAFS',
 'ELKJØP NORGE',
 'COOP MARKED',
 'EGON',
 'RINGO',
 'SKEIDAR',
 'CHOCO',
 'APOTEK 1 GRUPPEN',
 'DEKK PARTNER',
 'DE HISTORISKE HOTEL & SPISESTEDER',
 'BIG HORN STEAK HOUSE',
 'SPORT OUTLET',
 'ZOOKJEDEN BUDDY',
 'FRI B

In [61]:
chain_extra_grouped = chain_extra_notNa.groupby(["chain_name"], as_index = False)["store_id"].count().sort_values(by = ["store_id"], ascending = False)
chain_extra_grouped_list = chain_extra_grouped.chain_name.values.tolist()

chain_extra_grouped_list

['KIWI NORGE',
 'NILLE',
 'NARVESEN',
 'APOTEK 1 GRUPPEN',
 'NORSKE SHELL',
 'VINMONOPOLET',
 'COOP OBS!',
 'COOP PRIX',
 'COOP MARKED',
 'COOP EXTRA',
 'CIRCLE K DETALJIST',
 'VITUSAPOTEK',
 'JOKER',
 'EUROPRIS',
 'EURO SKO NORGE',
 'ESSO NORGE',
 'CUBUS',
 'UNO X NORGE',
 'BOOTS NORGE',
 'PRINCESS GRUPPEN',
 'DRESSMANN',
 'SEVEN-ELEVEN',
 'MIX',
 'VITA',
 'MENY',
 'INTERFLORA NORGE SA',
 'KID INTERIØR',
 'REMA FRANCHISE NORGE',
 'SPAR',
 'ARK BOKHANDEL',
 'MESTER GRØNN',
 'JERNIA DETALJ',
 'BIK BOK',
 'H & M HENNES & MAURITZ',
 'NYE NOTABENE',
 'LINDEX',
 'CARLINGS',
 'KAPPAHL',
 'SYNSAM',
 'NORLI LIBRIS',
 'COOP MEGA',
 'LIFE',
 'G-SPORT',
 'JYSK',
 'ELKJØP NORGE',
 'VERO MODA',
 'EXPERT',
 'RINGO',
 'GLITTER',
 'CLAS OHLSON',
 'YX NORGE',
 'COOP BYGGMIX',
 'KITCHN BUTIKKDRIFT',
 'COOP ANDRE',
 'JACK & JONES',
 'BRILLELAND',
 'PLANTASJEN NORGE',
 'KICKS NORGE',
 'DELI DE LUCA NORGE',
 'NIKITA',
 'MATCH',
 'KREMMERHUSET TING & SÅNT',
 'MCDONALDS',
 'MATKROKEN',
 'TELIA NORGE',
 'ENKL

In [21]:
#stores_gk_impuded2.to_csv("stores_gk_impuded.csv")
#stores_test_impuded2.to_csv("store_test_impuded.csv")

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name
